In [31]:
import pymongo as pm
import pandas as pd
import numpy as np

from zlib import DEF_BUF_SIZE
import dash
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_core_components as dcc
from matplotlib.pyplot import figure
import plotly.express as px
from dash.dependencies import Input, Output

import plotly.graph_objects as go
import warnings

In [32]:
def connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """
    try:
        if username and password:
            mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
            conn = pm.MongoClient(mongo_uri)
        else:
            conn = pm.MongoClient(host, port)
        return conn[db]
    except Exception as e:
        print("Mongo Connection error",e)
    finally:
        print("Mongo Connection Close")

In [33]:
from pymongo import MongoClient

# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
filter={
    'tool_type': 'SAST'
}

Sast_result = client['boman_dev']['Vulnerabilities'].find(
  filter=filter
)

In [34]:

def read_mongo(db, collection, query={}, host='127.0.0.1', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df

In [35]:
sast_result = read_mongo('boman_dev','Vulnerabilities',query={'tool_type': 'SAST'})
dast_result = read_mongo('boman_dev','Vulnerabilities',query={'tool_type': 'DAST'})
sca_result = read_mongo('boman_dev','Vulnerabilities',query={'tool_type': 'SCA'})
secret_scanner_results = read_mongo('boman_dev','Vulnerabilities',query={'tool_type': 'Secret Scanner'})
app = read_mongo('boman_dev','app')
vuln = read_mongo('boman_dev','Vulnerabilities')
results = read_mongo('boman_dev','result')

Mongo Connection Close
Mongo Connection Close
Mongo Connection Close
Mongo Connection Close
Mongo Connection Close
Mongo Connection Close
Mongo Connection Close


In [36]:
app.head(2)

,app_id,customer_id,app_token,app_name,type,language,app_type,has_login,database_present,user_interaction,running_env,created_at,updated_at
0,1,2,62d86c99-0107-46ec-8d8d-fc25205d76f3,Prod API v3,Web,Java,1,1,1,1,,2022-01-28 11:47:27,2022-02-23 16:03:40
1,2,1,d143e7b1-0d94-4f7f-b6cf-26ce28f2f4ff,SUMERU WEBAPP,Web,python,1,1,1,1,,2022-04-04 06:34:25,2022-04-05 03:02:54


In [37]:
vuln.head(2)

,vuln_id,scan_details_id,scan_token,scan_engine_version,tool_name,customer_id,vuln_name,vuln_severity,boman_severity,vuln_description,...,risk_desc,solution,ref,was_cid,source_id,parameter,attack,affected_url,port,method
0,183,143,e4349876-1a59-49f4-8218-2b8dcdcfe14a,NaN,OWASP Dependency Check,1,CVE-2018-14042,MEDIUM,MEDIUM,"In Bootstrap before 4.1.2, XSS is possible in ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,184,143,e4349876-1a59-49f4-8218-2b8dcdcfe14a,NaN,OWASP Dependency Check,1,CVE-2019-8331,MEDIUM,MEDIUM,In Bootstrap before 3.4.1 and 4.3.x before 4.3...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
vuln.columns

Index(['vuln_id', 'scan_details_id', 'scan_token', 'scan_engine_version',
       'tool_name', 'customer_id', 'vuln_name', 'vuln_severity',
       'boman_severity', 'vuln_description', 'file', 'file_path', 'cvss_v2',
       'cvss_v3', 'cwe', 'evidence', 'report_schema', 'report_date', 'notes',
       'reference', 'tool_type', 'customer_status', 'fixed_at', 'created_at',
       'updated_at', 'branch', 'commit', 'commit_hash', 'date', 'path',
       'reason', 'stringsFound', 'vuln_loc', 'affected_file',
       'vuln_confidence', 'test_id', 'false_positive', 'code', 'cwe_cve',
       'owasp', 'tool_version', 'checklist', 'more_info', 'errors',
       'ml_status', 'host_name', 'ssl_enabled', 'plugin_id', 'risk_code',
       'confidence', 'risk_desc', 'solution', 'ref', 'was_cid', 'source_id',
       'parameter', 'attack', 'affected_url', 'port', 'method'],
      dtype='object')

In [39]:
vuln.boman_severity.isna().sum()

733

In [41]:
app_wise_vuln_name = vuln.groupby(['vuln_name','boman_severity']).agg({'customer_id':'unique'})
app_wise_vuln_name.reset_index(inplace=True)
app_wise_vuln_name[0:5]

,vuln_name,boman_severity,customer_id
0,,MEDIUM,"[1, 0]"
1,Absence of Anti-CSRF Tokens,MEDIUM,"[1, 7]"
2,CVE-2010-5312,MEDIUM,[1]
3,CVE-2012-6662,MEDIUM,[1]
4,CVE-2012-6708,MEDIUM,"[1, 4]"


In [42]:
app_wise_vuln_name.app_id.value_counts()

AttributeError: 'DataFrame' object has no attribute 'app_id'

In [44]:
def split_multicolumn(col_series):
    result_df = col_series.to_frame()
    options = []
    # Iterate over the column
    for idx, value  in col_series[col_series.notnull()].iteritems():
        # Break each value into list of options
        for option in value.split(';'):
            # Add the option as a column to result
            if not option in result_df.columns:
                options.append(option)
                result_df[option] = False
            # Mark the value in the option column as True
            result_df.at[idx, option] = True
    return result_df[options]

In [45]:
vn = split_multicolumn(app_wise_vuln_name.vuln_name)

C:\Users\sumeruinfra\AppData\Local\Temp\ipykernel_42616\3580593157.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result_df[option] = False
C:\Users\sumeruinfra\AppData\Local\Temp\ipykernel_42616\3580593157.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result_df[option] = False
C:\Users\sumeruinfra\AppData\Local\Temp\ipykernel_42616\3580593157.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all 

In [46]:
severity = split_multicolumn(app_wise_vuln_name.boman_severity)
severity.head(2)

,MEDIUM,HIGH,LOW
0,True,False,False
1,True,False,False


In [47]:
severity_percentages = severity.mean().sort_values(ascending=False) * 100
severity_percentages

MEDIUM    58.878505
HIGH      32.710280
LOW        8.411215
dtype: float64

In [48]:
dev_type_totals = vn.sum().sort_values(ascending=False)
dev_type_totals[0:5]

Cross-Site Scripting     2
SQL Injection            2
Remote Code Execution    2
Mass Assignment          2
Dangerous Send           1
dtype: int64

In [49]:
result = pd.merge(sast_result, app, how='inner',on='customer_id')

In [50]:
result.head()

,vuln_id,scan_details_id,scan_token,tool_name,customer_id,vuln_name,vuln_loc,affected_file,vuln_description,vuln_severity,...,app_name,type,language,app_type,has_login,database_present,user_interaction,running_env,created_at_y,updated_at_y
0,439,1,93e567c0-16e1-47b8-be38-bb2669403bb9,Brakeman,1,Remote Code Execution,6,app/controllers/password_resets_controller.rb,`Marshal.load` called with parameter value,,...,SUMERU WEBAPP,Web,python,1,1,1,1,,2022-04-04 06:34:25,2022-04-05 03:02:54
1,439,1,93e567c0-16e1-47b8-be38-bb2669403bb9,Brakeman,1,Remote Code Execution,6,app/controllers/password_resets_controller.rb,`Marshal.load` called with parameter value,,...,Demo,Web,python,1,1,1,1,,2022-04-05 05:38:25,2022-04-05 05:38:25
2,439,1,93e567c0-16e1-47b8-be38-bb2669403bb9,Brakeman,1,Remote Code Execution,6,app/controllers/password_resets_controller.rb,`Marshal.load` called with parameter value,,...,Django nv,Web,Python,1,1,1,1,,2022-04-07 08:22:44,2022-04-08 14:11:33
3,439,1,93e567c0-16e1-47b8-be38-bb2669403bb9,Brakeman,1,Remote Code Execution,6,app/controllers/password_resets_controller.rb,`Marshal.load` called with parameter value,,...,ROR,API,php,4,1,1,0,,2022-04-07 10:05:53,2022-04-08 14:12:14
4,439,1,93e567c0-16e1-47b8-be38-bb2669403bb9,Brakeman,1,Remote Code Execution,6,app/controllers/password_resets_controller.rb,`Marshal.load` called with parameter value,,...,Railsgoat,Web,ruby,1,1,1,1,,2022-04-08 09:01:36,2022-04-08 09:01:36


In [51]:
def Repeated_vulnerabilities_across_all_applications_particular(customer_ids, vuln_severity):
    result = pd.merge(sast_result, app, on='customer_id')
    customer_id = result['customer_id'] == customer_ids
    vuln_severity = result['vuln_severity'] == vuln_severity
    vuln_names = result['vuln_name']
    top_high_vuln_severity = result.where(customer_id & vuln_severity)
    return top_high_vuln_severity

In [52]:
customer_1_High = Repeated_vulnerabilities_across_all_applications_particular(1,'High')
customer_1_Low = Repeated_vulnerabilities_across_all_applications_particular(1,'Low')
customer_1_Medium = Repeated_vulnerabilities_across_all_applications_particular(1,'Medium')

In [53]:
def repeated_vulnerabilities_severity(tool_name):
        rvs = tool_name.vuln_severity.value_counts()
        data = dict({'Vulnerabilities Severity':list(rvs.keys()),
                        "Severity Count":list(rvs.values)})
        return data

In [54]:
repeated_vulnerabilities_severity(sast_result)

{'Vulnerabilities Severity': ['', 'LOW', '5', 'HIGH', 'MEDIUM', 'INFO'],
 'Severity Count': [708, 183, 68, 63, 63, 2]}

In [55]:
## SAST, DAST, SCA ( Top Vul Acros all the application)

In [56]:
# SAST Vulnerabilities Severity
sast_vuln_severity_count = sast_result.boman_severity.value_counts()
# DAST Vulnerabilities Severity
dast_vuln_severity_count = dast_result.boman_severity.value_counts()
# SCA Vulnerabilities Severity
sca_vuln_severity_count = sca_result.boman_severity.value_counts()

In [57]:
# SAST Vulnerabilities Severity
sast_vuln_severity = sast_result.boman_severity.unique()
sast_vn = sast_result.vuln_name.unique()

In [58]:
sast_vuln_severity

array([None, 'MEDIUM', 'HIGH', 'LOW'], dtype=object)

In [59]:
X = vuln.created_at.dt.to_period('Q').sort_index().value_counts()
X.index = X.index.to_timestamp()
X = X.sort_index()
X[X.index] = np.cumsum(X.values)

rolling = vuln.created_at.dt.to_period('M').sort_index().value_counts()
rolling.index = rolling.index.to_timestamp()
rolling = rolling.sort_index()
rolling[rolling.index] = np.cumsum(rolling.values)
rolling = rolling.rolling(12, axis=0).sum().pct_change(axis=0)

AttributeError: Can only use .dt accessor with datetimelike values

#### App wise Vuln Names Repreted

In [60]:
sast_app_vuln_names_severitys_medium = read_mongo('boman_dev','Vulnerabilities',query={'customer_id':1,'tool_type':'SAST','customer_status':1,'boman_severity':'MEDIUM'})
sast_app_vuln_names_severitys_medium

Mongo Connection Close


,vuln_id,scan_details_id,scan_token,tool_name,customer_id,vuln_name,vuln_loc,affected_file,vuln_description,vuln_severity,...,tool_version,checklist,more_info,errors,tool_type,ml_status,customer_status,fixed_at,created_at,updated_at
0,943,179,cf26b2e7-f1be-45b0-bc9b-b75dce51af34,Bandit,1,hardcoded sql expressions,"184,185",/src/taskManager/views.py,Possible SQL injection vector through string-b...,MEDIUM,...,,,,,SAST,1,1,2022-04-05 03:37:47,2022-04-05 02:58:39,2022-04-08 11:14:10


In [61]:
sast_app_vuln_names_severitys_low = read_mongo('boman_dev','Vulnerabilities',query={'customer_id':1,'tool_type':'SAST','customer_status':1,'boman_severity':'LOW'})
sast_app_vuln_names_severitys_low

Mongo Connection Close


,vuln_id,scan_details_id,scan_token,tool_name,customer_id,vuln_name,vuln_loc,affected_file,vuln_description,vuln_severity,...,tool_version,checklist,more_info,errors,tool_type,ml_status,customer_status,fixed_at,created_at,updated_at
0,872,141,e4349876-1a59-49f4-8218-2b8dcdcfe14a,Bandit,1,hardcoded password string,751,/src/taskManager/views.py,Possible hardcoded password: '',LOW,...,,,,,SAST,1,1,2022-04-04 13:33:33,2022-04-04 13:29:52,2022-04-05 04:56:37
1,942,179,cf26b2e7-f1be-45b0-bc9b-b75dce51af34,Bandit,1,hardcoded password string,751,/src/taskManager/views.py,Possible hardcoded password: '',LOW,...,,,,,SAST,1,1,2022-04-05 04:32:42,2022-04-05 02:58:39,2022-04-08 11:14:07
2,966,196,9c5a8f43-70da-4fd8-ae40-cdc56e4a1e65,Bandit,1,hardcoded password string,"24,25,26,27",/src/taskManager/settings.py,Possible hardcoded password: '0yxzudryd8)-%)(f...,LOW,...,,,,,SAST,1,1,2022-04-05 05:57:07,2022-04-05 05:40:55,2022-04-08 11:52:51
3,2,1,2fb73968-3c64-4505-bc35-bd819cb4727e,Bandit,1,hardcoded password string,"24,25,26,27",/src/taskManager/settings.py,Possible hardcoded password: '0yxzudryd8)-%)(f...,LOW,...,,,,,SAST,1,1,2022-05-13 09:58:46,2022-05-13 09:57:10,2022-05-13 10:33:08


In [62]:
sast_app_vuln_names_severitys_high = read_mongo('boman_dev','Vulnerabilities',query={'customer_id':1,'tool_type':'SAST','customer_status':1,'boman_severity':'HIGH'})
sast_app_vuln_names_severitys_high

Mongo Connection Close


,vuln_id,scan_details_id,scan_token,tool_name,customer_id,vuln_name,vuln_loc,affected_file,vuln_description,vuln_severity,...,tool_version,checklist,more_info,errors,tool_type,ml_status,customer_status,fixed_at,created_at,updated_at
0,927,169,123fd6ec-f6a8-4e95-b007-010b12bcffed,Bandit,1,start process with a shell,"33,34,35,36,37,38,39",/src/taskManager/misc.py,"Starting a process with a shell, possible inje...",HIGH,...,,,,,SAST,1,1,2022-04-04 15:57:51,2022-04-04 15:53:55,2022-04-07 06:03:51
1,1,1,2fb73968-3c64-4505-bc35-bd819cb4727e,Bandit,1,start process with a shell,"33,34,35,36,37,38,39",/src/taskManager/misc.py,"Starting a process with a shell, possible inje...",HIGH,...,,,,,SAST,1,1,2022-05-13 09:59:53,2022-05-13 09:57:10,2022-05-13 10:33:08


In [63]:
dast_vuln_names_severity_medium_customer = read_mongo('boman_dev','Vulnerabilities',query={'customer_id':1,'tool_type':'DAST','customer_status':1,'boman_severity':'MEDIUM'})
dast_vuln_names_severity_medium_customer.head()

Mongo Connection Close


,vuln_id,scan_details_id,scan_token,tool_name,customer_id,vuln_name,host_name,ssl_enabled,plugin_id,risk_code,...,port,method,more_info,tool_type,customer_status,fixed_at,false_positive,ml_status,created_at,updated_at
0,1,6,43e772fc-b3c3-4acd-8ff2-18b9f7b48586,OWASP Zap Baseline,1,Absence of Anti-CSRF Tokens,demo.testfire.net,false,10202,2,...,80,GET,,DAST,1,2022-05-13 10:11:49,0,1,2022-05-13 10:08:56,2022-05-13 10:34:30
1,624,222,e26e19e1-d235-4ead-848d-49a9c880c13d,OWASP Zap Baseline,1,Absence of Anti-CSRF Tokens,demo.testfire.net,false,10202,2,...,80,GET,,DAST,1,2022-06-03 07:24:36,0,1,2022-06-03 07:13:25,2022-06-03 07:24:35
2,762,156,e3ace008-b02a-472b-9fbb-36e28b806650,OWASP Zap Baseline,1,Absence of Anti-CSRF Tokens,demo.testfire.net,false,10202,2,...,80,GET,,DAST,1,2022-04-04 14:04:13,0,1,2022-04-04 14:03:19,2022-04-05 05:23:20


In [64]:
dast_vuln_names_severity_low_customer = read_mongo('boman_dev','Vulnerabilities',query={'customer_id':1,'tool_type':'DAST','customer_status':1,'boman_severity':'Low'})
dast_vuln_names_severity_low_customer.head()

Mongo Connection Close


KeyError: '_id'

In [ ]:
sca_result.boman_severity.value_counts()

MEDIUM    830
HIGH       60
null       33
LOW         3
Name: boman_severity, dtype: int64

In [ ]:
sca_vuln_names_severity_low_customer = read_mongo('boman_dev','Vulnerabilities',query={'tool_type':'SCA','customer_status':1,'boman_severity':'LOW'})
sca_vuln_names_severity_low_customer.head()

In [65]:
all_vuln_names_severitys_medium = read_mongo('boman_dev','Vulnerabilities',query={'customer_id':1,'customer_status':1,'boman_severity':'MEDIUM'})
all_vuln_names_severitys_medium

Mongo Connection Close


,vuln_id,scan_token,scan_details_id,tool_name,vuln_name,branch,commit,commit_hash,date,path,...,ref,cwe,was_cid,source_id,parameter,attack,evidence,affected_url,port,method
0,148,123fd6ec-f6a8-4e95-b007-010b12bcffed,172,Trufflehog,High Entropy found in commit Added missing use...,origin/master,Added missing user auth files\n,22a8d8d18bf3a4eeaba8f83a50afc212b657d31c,2015-04-04 00:11:37,taskManager/forms.py,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,943,cf26b2e7-f1be-45b0-bc9b-b75dce51af34,179,Bandit,hardcoded sql expressions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,43e772fc-b3c3-4acd-8ff2-18b9f7b48586,6,OWASP Zap Baseline,Absence of Anti-CSRF Tokens,NaN,NaN,NaN,NaN,NaN,...,<p>http://projects.webappsec.org/Cross-Site-Re...,352,9,1,,,"<form id=""frmSearch"" method=""get"" action=""/sea...",http://demo.testfire.net,80.0,GET
3,624,e26e19e1-d235-4ead-848d-49a9c880c13d,222,OWASP Zap Baseline,Absence of Anti-CSRF Tokens,NaN,NaN,NaN,NaN,NaN,...,<p>http://projects.webappsec.org/Cross-Site-Re...,352,9,1,,,"<form id=""frmSearch"" method=""get"" action=""/sea...",http://demo.testfire.net/feedback.jsp,80.0,GET
4,762,e3ace008-b02a-472b-9fbb-36e28b806650,156,OWASP Zap Baseline,Absence of Anti-CSRF Tokens,NaN,NaN,NaN,NaN,NaN,...,<p>http://projects.webappsec.org/Cross-Site-Re...,352,9,1,,,"<form id=""frmSearch"" method=""get"" action=""/sea...",http://demo.testfire.net,80.0,GET


In [66]:
all_vuln_names_severitys_low= read_mongo('boman_dev','Vulnerabilities',query={'customer_id':1,'customer_status':1,'boman_severity':'LOW'})
all_vuln_names_severitys_low

Mongo Connection Close


,vuln_id,scan_details_id,scan_token,tool_name,customer_id,vuln_name,vuln_loc,affected_file,vuln_description,vuln_severity,...,tool_version,checklist,more_info,errors,tool_type,ml_status,customer_status,fixed_at,created_at,updated_at
0,872,141,e4349876-1a59-49f4-8218-2b8dcdcfe14a,Bandit,1,hardcoded password string,751,/src/taskManager/views.py,Possible hardcoded password: '',LOW,...,,,,,SAST,1,1,2022-04-04 13:33:33,2022-04-04 13:29:52,2022-04-05 04:56:37
1,942,179,cf26b2e7-f1be-45b0-bc9b-b75dce51af34,Bandit,1,hardcoded password string,751,/src/taskManager/views.py,Possible hardcoded password: '',LOW,...,,,,,SAST,1,1,2022-04-05 04:32:42,2022-04-05 02:58:39,2022-04-08 11:14:07
2,966,196,9c5a8f43-70da-4fd8-ae40-cdc56e4a1e65,Bandit,1,hardcoded password string,"24,25,26,27",/src/taskManager/settings.py,Possible hardcoded password: '0yxzudryd8)-%)(f...,LOW,...,,,,,SAST,1,1,2022-04-05 05:57:07,2022-04-05 05:40:55,2022-04-08 11:52:51
3,2,1,2fb73968-3c64-4505-bc35-bd819cb4727e,Bandit,1,hardcoded password string,"24,25,26,27",/src/taskManager/settings.py,Possible hardcoded password: '0yxzudryd8)-%)(f...,LOW,...,,,,,SAST,1,1,2022-05-13 09:58:46,2022-05-13 09:57:10,2022-05-13 10:33:08


In [67]:
all_vuln_names_severitys_high= read_mongo('boman_dev','Vulnerabilities',query={'customer_id':1,'customer_status':1,'boman_severity':'HIGH'})
all_vuln_names_severitys_high

Mongo Connection Close


,vuln_id,scan_details_id,scan_token,tool_name,customer_id,vuln_name,vuln_loc,affected_file,vuln_description,vuln_severity,...,tool_version,checklist,more_info,errors,tool_type,ml_status,customer_status,fixed_at,created_at,updated_at
0,927,169,123fd6ec-f6a8-4e95-b007-010b12bcffed,Bandit,1,start process with a shell,"33,34,35,36,37,38,39",/src/taskManager/misc.py,"Starting a process with a shell, possible inje...",HIGH,...,,,,,SAST,1,1,2022-04-04 15:57:51,2022-04-04 15:53:55,2022-04-07 06:03:51
1,1,1,2fb73968-3c64-4505-bc35-bd819cb4727e,Bandit,1,start process with a shell,"33,34,35,36,37,38,39",/src/taskManager/misc.py,"Starting a process with a shell, possible inje...",HIGH,...,,,,,SAST,1,1,2022-05-13 09:59:53,2022-05-13 09:57:10,2022-05-13 10:33:08


In [68]:
# Year Wise
by_year = pd.to_datetime(dast_result['created_at']).dt.strftime('%Y').value_counts().sort_index()
#by_year.index = pd.PeriodIndex(by_year.index)
by_year = by_year.rename_axis('Yearly').reset_index(name='counts')

# quarter Wise
by_quarter = pd.to_datetime(dast_result['created_at']).dt.to_period('Q').value_counts().sort_index()
by_quarter.index = pd.PeriodIndex(by_quarter.index)
by_quarter = by_quarter.rename_axis('quarter').reset_index(name='counts')

# Month Wise
by_month = pd.to_datetime(dast_result['created_at']).dt.strftime('%m').value_counts().sort_index()
by_month = by_month.rename_axis('month').reset_index(name='counts')

# Weekly wise
by_weak = pd.to_datetime(dast_result['created_at']).dt.strftime('%A').value_counts().sort_index()
#by_weak.index = pd.PeriodIndex(by_weak.index)
by_weak = by_weak.rename_axis('weekly').reset_index(name='counts')

# Day wise
by_day = pd.to_datetime(dast_result['created_at']).dt.strftime('%d').value_counts().sort_index()
#by_day.index = pd.PeriodIndex(by_day.index)
by_day = by_day.rename_axis('Days').reset_index(name='counts')

trands_plot = [ 
html.H2('Live Dynamic Application Security Testing (DAST)  Trend Analysis',style={'textAlign':'center'}),
html.Hr(),
#html.H4("SAST Yearly  Issues Creations Trends"),
dcc.Graph(id = "yearly_count",
            figure=px.line(x=by_year['Yearly'].astype(dtype=str),
                                                y=by_year['counts'],
                                                labels={'x':'Yearly', 'y':'Count'},
                                                markers=True,
                                                title='DAST Yearly  Issues Creations Trends',
                                                color_discrete_sequence=px.colors.cyclical.HSV),

            config= {'displaylogo': False},
        ),
#html.H4("SAST Quarterly Issues Created"),
dcc.Graph(id='quarter_count',
        config= {'displaylogo': False},
        figure= px.line(x=by_quarter['quarter'].astype(dtype=str), 
                y=by_quarter['counts'],
                markers= True,
                title = 'DAST Quarterly Issues Created',
                labels = {'x':'Quarterly Issues','y':'Count'},
                color_discrete_sequence=px.colors.sequential.Cividis)
        ),
#html.H4("SAST Monthly Issues Created"),
dcc.Graph(id='month_count',
        config= {'displaylogo': False},
        figure= px.line(x=by_month['month'].astype(dtype=str), 
                y=by_month['counts'],
                markers= True,
                title = 'DAST Monthly Issues Created',
                labels = {'x':'Monthly Issues','y':'Count'},
                color_discrete_sequence=px.colors.sequential.Magenta)
        ),
#html.H4("SAST Weekly Issues Created"),
dcc.Graph(id='weekly_count',
        config= {'displaylogo': False},
        figure= px.line(x=by_weak['weekly'].astype(dtype=str), 
                y=by_weak['counts'],
                markers= True,
                title = 'DAST Weekly Issues Created',
                labels = {'x':'Weekly Issues','y':'Count'},
                color_discrete_sequence=px.colors.sequential.Electric),
        ),
#html.H4("SAST Daily Issues Created"),
dcc.Graph(id='daily_count',
        config= {'displaylogo': False},
        figure= px.line(by_day,
                        x='Days',
                        y='counts',
                        title = "DAST Daily Issues Created",
                        labels = {'Days':'Daily Issues','counts':'Count'},
                        color_discrete_sequence= px.colors.sequential.Bluered)),
]

In [69]:
sast_result.columns

Index(['vuln_id', 'scan_details_id', 'scan_token', 'tool_name', 'customer_id',
       'vuln_name', 'vuln_loc', 'affected_file', 'vuln_description',
       'vuln_severity', 'boman_severity', 'vuln_confidence', 'test_id',
       'false_positive', 'code', 'cwe_cve', 'owasp', 'tool_version',
       'checklist', 'more_info', 'errors', 'tool_type', 'ml_status',
       'customer_status', 'fixed_at', 'created_at', 'updated_at'],
      dtype='object')

In [70]:
df = vuln[['tool_type','tool_name','boman_severity','vuln_name']]
df.isnull().sum()

tool_type           0
tool_name           0
boman_severity    733
vuln_name           0
dtype: int64

In [71]:
fig = px.parallel_categories(df, 
                        title = 'Total Repeated Vulnerabilities Names with Severity Level',
                        labels = {'tool_type':'Tool Type','boman_severity':'Severity','vuln_name':'Vulnerability Names'},
                        color_continuous_scale = px.colors.diverging.balance,)
fig.show()

In [73]:
vn = vuln['vuln_name']
vs = vuln['vuln_severity']

In [74]:
from collections import defaultdict

word_dict = defaultdict(int)
for word in vn:
    word_dict[word] +=1

vuln_repreted_count = pd.Series(word_dict)
vuln_repreted_count.head()

CVE-2018-14042    26
CVE-2019-8331     26
CVE-2015-9251     81
CVE-2018-14040    26
CVE-2019-11358    82
dtype: int64

In [81]:
vs_dict = defaultdict(int)
for word in vs:
    vs_dict[word] +=1

vuln_severity_repreted_count = pd.Series(vs_dict)
vuln_severity_repreted_count.head()

MEDIUM      861
medium       26
HIGH        123
CRITICAL      3
Unknown       3
dtype: int64

In [79]:
df = pd.DataFrame()
df.head()

,CVE-2018-14042,CVE-2019-8331,CVE-2015-9251,CVE-2018-14040,CVE-2019-11358,CVE-2012-6708,CVE-2018-14041,CVE-2016-10735,jquery-migrate bug: 11290,CVE-2012-6662,...,MEDIUM,medium,HIGH,CRITICAL,Unknown,LOW,low,NaN,INFO,5
0,26.0,26.0,81.0,26.0,82.0,55.0,26.0,26.0,25.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,861.0,26.0,123.0,3.0,3.0,186.0,1.0,2077.0,2.0,68.0
